In [16]:
import numpy as np
import pandas as pd

def main():
    train_df = pd.read_csv('quasar_train.csv', header = 0)
    test_df = pd.read_csv('quasar_test.csv', header = 0)
    
    train = train_df.as_matrix()
    test = test_df.as_matrix()
    
    print(train.shape)
    print(test.shape)
    
if __name__ == "__main__":
    main()
    

(200, 450)
(50, 450)
